In [12]:
# Импортируем все нужные библиотеки
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import pandas as pd
import gdown

Начало работы с уже обработанными файлами

1. Загрузим рейтинги

In [14]:
gdown.download('https://drive.google.com/uc?id=1mKBwlwo1HVAIFljJ7FyncyqOo2d9DMsW', output='fortune5000_2023.xlsx', quiet=True)
gdown.download('https://drive.google.com/uc?id=1vPP8sHjxHipW5Cc6B6rhXmzIyJfGVIxM', output='fortune5000_2000.xlsx', quiet=True)


'fortune5000_2000.xlsx'

In [15]:
df_2000 = pd.read_excel('fortune5000_2000.xlsx')
df_2000 = df_2000.rename(columns = {'NAME' : '2000', 'RANK' : 'Rank'})
df_2000.head()

,Rank,2000,REMOVE\nREVENUES
0,1,"Wal-Mart Stores, Inc.","24,652,500"
1,2,General Motors Corporation,"18,676,300"
2,3,Exxon Mobil Corporation,"18,246,600"
3,4,Ford Motor Company,"16,363,000"
4,5,General Electric Company,"13,169,800"


In [16]:
df = pd.read_excel('fortune5000_2023.xlsx')
df = df.rename(columns = {'NAME' : '2023', 'RANK' : 'Rank_2023'})
df = df[['Rank_2023','2023']]
df.head()

,Rank_2023,2023
0,1,Walmart
1,2,Amazon
2,3,Exxon Mobil
3,4,Apple
4,5,UnitedHealth Group


Объединим рейтинги в одну табличку

In [17]:
df['2000'] = df_2000['2000']
df.head()

,Rank_2023,2023,2000
0,1,Walmart,"Wal-Mart Stores, Inc."
1,2,Amazon,General Motors Corporation
2,3,Exxon Mobil,Exxon Mobil Corporation
3,4,Apple,Ford Motor Company
4,5,UnitedHealth Group,General Electric Company


Итак, мы скачали два рейтинга Fortune global 1000.


---


Теперь нам нужно привести все названия компаний к одному времени (ведь некоторые компании в 2000 году назывались по-другому.)

Для этого
*   Для каждой компании из рейтинга 2023 года найдем список ее предыдущих названий
*   При сравнении рейтингов будем сравнивать эти найденные названия с названиями компаний рейтинга 2000 года.





In [43]:
# загрузим файл с историей названий компаний из рейтинга 2023.
with open('chat_gpt_helper.txt', "r") as file:
    full_company_names_2023_history = json.load(file)
def get_names(company, full_company_names_2023_history):
    list_names = None
    for i in full_company_names_2023_history:
        if company in i:
            list_names = i[company]
            if len(list_names) == 0:
                print(company, 'IsNone')
            return list_names
f'Пример - у компании Unum Group вот такие были названия: ' + str(get_names('Unum Group', full_company_names_2023_history))

"Пример - у компании Unum Group вот такие были названия: ['Unum Group', 'Unum Provident Corporation']"

Итак, вот основная программа : для каждой фирмы из рейтинга Global 2023 попробуем найти совпадение по имени в рейтинге 2003 года, при этом проведем учтем, что в 2003 году компания могла называться по-другому, для этого также будем искать совпадения в именах компании в прошлом

In [40]:
fg = []
for company in df['2023']:
    all_names_of_the_company = get_names(company, full_company_names_2023_history)
    if company in company_previous_names1:
        all_names_of_the_company.append(company_previous_names1[company])
    if company in company_previous_names:
        all_names_of_the_company.append(company_previous_names[company])
    try:
        for ig in all_names_of_the_company:
            if ig in list(df['2000']):
                fg.append(ig)
    except:
        print(company)
f'Количество компаний, находящихся в обоих рейтингах = {len(fg)}'

'Количество компаний, находящихся в обоих рейтингах = 341'

Итак, в ходе исследования выяснено, что 341 компания осталась в рейтинге, то есть 34.1 %